In [2]:
import pandas as pd
from Bio import SeqIO
import glob
def read_fasta_to_list(f):
	myList = []
	for r in SeqIO.parse(f, "fasta"):
		myList.append(str(r.seq).upper())
	return myList	
def revcomp(seq):
	try: ## python2
		tab = string.maketrans(b"ACTG", b"TGAC")
	except:  ## python3
		tab = bytes.maketrans(b"ACTG", b"TGAC")
	return seq.translate(tab)[::-1]
# input=sys.argv[1]
# gRNA_list = sys.argv[2]



def is_gRNA(seq,gRNA):
	if gRNA in seq:
		return 1
	if revcomp(gRNA) in seq:
		return 1
	return 0

def get_value(input,gRNA_list):
	df = pd.DataFrame(read_fasta_to_list(input))
	gRNA = pd.read_csv(gRNA_list,sep="\t",header=None)
	for i,r in gRNA.iterrows():
		df[r[0]] = [is_gRNA(seq,r[1]) for seq in df[0]]

	total = df.shape[0]
	df2 = df.copy()
	df2 = df2.drop([0],axis=1)
	df3 = pd.DataFrame((df2.sum()/total)*100)
	df3.columns = [f'%Read_{input}']
	return df3


In [3]:
files = glob.glob("*FQ2FA.fa")
gRNA_file="gRNA_to_check.tsv"
df_list = []
for f in files:
    df_list.append(get_value(f,gRNA_file))
df = pd.concat(df_list,axis=1)
df

,%Read_VK2341_S3.FQ2FA.fa,%Read_VK2348_S10.FQ2FA.fa,%Read_VK2339_S1.FQ2FA.fa,%Read_VK2340_S2.FQ2FA.fa,%Read_VK2347_S9.FQ2FA.fa,%Read_VK2343_S5.FQ2FA.fa,%Read_VK2346_S8.FQ2FA.fa,%Read_VK2345_S7.FQ2FA.fa,%Read_VK2344_S6.FQ2FA.fa,%Read_VK2342_S4.FQ2FA.fa
DNMT3A,49.995730,68.276284,2.290076,52.827512,61.450574,64.04157,57.587354,55.676734,1.646264,55.054521
TRAC1,0.017081,0.017946,0.000000,0.022572,0.021065,0.01671,0.023720,0.028018,0.000000,0.016557
TRAC_Control,0.000000,0.000137,0.000000,0.000000,0.000000,0.00000,0.000000,0.000301,0.000000,0.000000


In [4]:
df.to_csv("percent_reads_gRNA.csv")